In [194]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import pickle



df_events = pd.read_csv('preprocessed_data.csv',index_col=0)
df_events = df_events.drop_duplicates(subset=['venue','event','date','age','price','location','distance_miles'])
event_embeddings = df_events.embedding.apply(eval).apply(np.array)
event_embeddings = np.array(list(event_embeddings))
# Cache user_embedding using Pickle
with open('event_embeddings.pkl', 'wb') as f:
    pickle.dump(event_embeddings, f)


In [131]:
interests = ['Live music', 'DJ performances', 'Dancing', 'Karaoke', 'Comedy shows', 'Trivia nights', 'Art exhibits', 'Film screenings', 'Wine tasting', 'Craft beer tasting', 'Mixology classes', 'Food events', 'Sports viewing parties', 'Networking events', 'Tech events']
venue_types = ['Speakeasy Bar', 'Dive Bar', 'Nightclubs', 'Music Venue', 'Comedy Club', 'Theaters', 'Art Gallery', 'Film theaters', 'Wineries', 'Breweries', 'Cocktail Bars', 'Sports Bars', 'Restaurants with Live Music', 'Rooftop bars', 'Convention Centers']
price_range_pref = ['free', '$10-$20', '$20-$50', '$50-$100', 'over $100']
time_of_day_pref = ['daytime', 'evening', 'night']
neighborhoods_of_interest = ['Financial District', 'Nob Hill', 'Pacific Heights', 'Russian Hill', 'North Beach', 'The Mission', 'Castro', 'Haight-Ashbury', 'SoMa', 'Marina']

user_interests = ['Live music', 'DJ performances', 'Comedy shows', 'Food events', 'Networking events', 'Tech events']
user_venues = ['Speakeasy Bar', 'Dive Bar','Music Venue', 'Comedy Club', 'Cocktail Bars', 'Theaters']
user_prices = ['free', '$10-$20', '$20-$50']
user_time = ['evening', 'night']
user_neighborhoods = ['Nob Hill', 'Pacific Heights', 'Russian Hill', 'North Beach', 'The Mission', 'Castro', 'Haight-Ashbury']

In [132]:
import pandas as pd
import tiktoken
from openai.embeddings_utils import get_embedding
import os
import openai

openai.apikey = os.getenv("OPENAI_API_KEY")
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 5000  # the maximum for text-embedding-ada-002 is 8191

In [133]:
encoding = tiktoken.get_encoding(embedding_encoding)
combined = ("Event Preferences: " + str(user_interests) +  "; Venue Preferences: " + str(user_venues) + 
    "; Price Range Preferences: " + str(user_prices) +  "; Time of Day Preferences: " + str(user_time) + 
     "; Neighborhoods of Interest: " + str(user_neighborhoods))

user_embedding = get_embedding(combined, engine=embedding_model)
user_embedding = np.array(user_embedding)

In [189]:
import pickle

# Cache user_embedding using Pickle
with open('user_embedding.pkl', 'wb') as f:
    pickle.dump(user_embedding, f)

In [135]:
similarity_matrix = cosine_similarity(user_embedding.reshape(1, -1), event_embeddings)

## Load into Database

In [180]:
df_events = df_events.assign(similarity=similarity_matrix[0])

In [181]:
database_df = df_events[['venue', 'event', 'date', 'age', 'price', 'location',
       'venue_description', 'event_description', 'neighborhood', 'distance_miles', 
       'similarity']]

In [182]:
database_df = database_df.drop_duplicates(subset=['venue','event','date','age','price','location','distance_miles'])

In [183]:
import pymysql
import os

mysql_pass = os.getenv("MYSQL_PASSWORD")

# Connect to the MySQL server
conn = pymysql.connect(host='localhost', user='root', password=mysql_pass, db='mysfevents')
cursor = conn.cursor()


table_name = 'events'

drop_query = f"DROP TABLE IF EXISTS {table_name};"

# Execute the SQL query to drop the table if it exists
cursor.execute(drop_query)

# Commit the changes to the database
conn.commit()

cursor.close()
conn.close()

In [184]:
conn = pymysql.connect(host='localhost', user='root', password=mysql_pass, db='mysfevents')
cursor = conn.cursor()

create_table_query = """CREATE TABLE events (
                        id INT AUTO_INCREMENT PRIMARY KEY,
                        venue VARCHAR(150),
                        event VARCHAR(150),
                        date VARCHAR(150),
                        age VARCHAR(150),
                        price VARCHAR(150),
                        location VARCHAR(150),
                        venue_desc VARCHAR(3000),
                        event_desc VARCHAR(5000),
                        neighborhood VARCHAR(200),
                        distance FLOAT,
                        similarity FLOAT
                        );"""
cursor.execute(create_table_query)
conn.commit()

cursor.close()
conn.close()

In [185]:
database_df = database_df.fillna(value='')

In [186]:
sql_query = "INSERT INTO events (venue, event, date, age, price, location, venue_desc, event_desc, neighborhood, distance, similarity) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

data = database_df.to_records(index=False).tolist()


In [187]:
conn = pymysql.connect(host='localhost', user='root', password=mysql_pass, db='mysfevents')
cursor = conn.cursor()

cursor.executemany(sql_query, data)

conn.commit()

cursor.close()
conn.close()
